# Data analysis

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from urllib.parse import urlparse
import urllib
import json
import pandas as pd
import os

driver = webdriver.Chrome(ChromeDriverManager().install())

In [ ]:
driver.get("https://sarc.asystom.com/monitor/login")

elem = driver.find_element_by_name("user")
elem.clear()
elem.send_keys(os.environ["SITE_USER"])

elem = driver.find_element_by_name("password")
elem.clear()
elem.send_keys(os.environ["SITE_PASSWORD"])

btn = driver.find_element_by_class_name("css-mij01a-button")
btn.click()

In [ ]:
def get_dfs(parsed_json):
    dfs = []
    # parsed_json["results"][0]["series"][0]["columns"]
    # parsed_json["results"][0]["series"][0]["values"]

    for result in parsed_json["results"]:
        print("Result", result["statement_id"])
        for serie in result["series"]:
            df = pd.DataFrame(serie["values"], columns=serie["columns"])
            df.name = serie["name"]
            dfs.append(df)
    return dfs

In [ ]:
from urllib import parse
q="SELECT%20median(%22drift%22)%20FROM%20(SELECT%2020*log10(%22vib_x_vel%22*%22vib_y_vel%22*%22vib_z_vel%22)%20AS%20%22drift%22%20FROM%20%22Signature_Drift%22%20WHERE%20%22device%22%3D%2700-04-a3-0b-00-e9-b5-05%27%20AND%20time%20%3E%3D%20now()%20-%2030d%20AND%20%22mileage%22%252%3D1)%20GROUP%20BY%20time(2h)%20fill(none)%3BSELECT%20moving_average(%22decimated%22%2C%2050)%20FROM%20(SELECT%20median(%22drift%22)%20AS%20%22decimated%22%20FROM%20(SELECT%2020*log10(%22vib_x_vel%22*%22vib_y_vel%22*%22vib_z_vel%22)%20AS%20%22drift%22%20FROM%20%22Signature_Drift%22%20WHERE%20%22device%22%3D%2700-04-a3-0b-00-e9-b5-05%27%20AND%20time%20%3E%3D%20now()%20-%2030d%20AND%20%22mileage%22%252%3D1)%20GROUP%20BY%20time(1m)%20fill(none))"

q="https://sarc.asystom.com/monitor/api/datasources/proxy/8/query?db=asystom_db&q=SELECT%20last(%22drift%22)%20FROM%20(SELECT%2020*log10(%22vib_x_vel%22*%22vib_y_vel%22*%22vib_z_vel%22)%20AS%20%22drift%22%20FROM%20%22Signature_Drift%22%20WHERE%20%22device%22%3D%2700-04-a3-0b-00-e9-b5-05%27%20AND%20time%20%3E%3D%20now()%20-%201d%20AND%20%22mileage%22%252%3D1)%0A&epoch=ms"
q_clean = parse.unquote(q)
q_clean

In [ ]:
# SELECT vib_x_vel, vib_y_vel, vib_z_vel

delay = "200d"

q_clean = f"""
SELECT *
FROM "Signature_Drift"
WHERE "device"=\'00-04-a3-0b-00-e9-b5-05\' AND time >= now() - {delay};

SELECT *
FROM "Signature"
WHERE "device"=\'00-04-a3-0b-00-e9-b5-05\' AND time >= now() - {delay};


SELECT *
FROM "Temperature"
WHERE "device"=\'00-04-a3-0b-00-e9-b5-05\' AND time >= now() - {delay};


SELECT *
FROM "Current_Loop"
WHERE "device"=\'00-04-a3-0b-00-e9-b5-05\' AND time >= now() - {delay};
"""

q_econded = parse.quote(q_clean)
url = f'https://sarc.asystom.com/monitor/api/datasources/proxy/8/query?db=asystom_db&q={q_econded}&epoch=ms'

driver.get(url)
driver.page_source
content = driver.find_element_by_tag_name('pre').text
parsed_json = json.loads(content)
parsed_json
dfs = get_dfs(parsed_json)

In [ ]:
with open("all_data.json", "w") as f:
    json.dump(parsed_json, f)

In [ ]:
print(dfs[0].name)
dfs[0]

In [ ]:
print(dfs[1].name)
dfs[1]

In [ ]:
print(dfs[2].name)
dfs[2]

In [ ]:
print(dfs[3].name)
dfs[3]